In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pandas.io.json import json_normalize
from json import dumps

pd.set_option('display.max_colwidth', 105)

In [3]:
df = pd.read_json('episodes.json')
df = df['episodes']
df = json_normalize(df)

In [4]:
df.iloc[30:40]

,episodeAirDate,episodeDescription,episodeLink,episodeNum,episodeTitle,openingSequenceLocations,scenes,seasonNum
30,2014-04-06,"Tyrion welcomes a guest to King's Landing. At Castle Black, Jon stands trial. Daenerys is pointed to ...",/title/tt2816136/,1,Two Swords,"[King's Landing, Dragonstone, Dreadfort, Winterfell, The Wall, Meereen]","[{'sceneStart': '0:00:07', 'sceneEnd': '0:01:47', 'location': 'The Crownlands', 'subLocation': 'King'...",4
31,2014-04-13,Joffrey and Margaery's wedding has come. Tyrion breaks up with Shae. Ramsay tries to prove his worth ...,/title/tt2832378/,2,The Lion and the Rose,"[King's Landing, Dragonstone, Dreadfort, Winterfell, The Wall, Meereen]","[{'sceneStart': '0:01:54', 'sceneEnd': '0:04:15', 'location': 'The North', 'subLocation': 'The Dreadf...",4
32,2014-04-20,Tyrion is arrested for the murder of Joffrey and awaits trial. Sansa escapes King's Landing. Sam send...,/title/tt2972426/,3,Breaker of Chains,"[King's Landing, Dragonstone, Dreadfort, Winterfell, The Wall, Meereen]","[{'sceneStart': '0:01:55', 'sceneEnd': '0:02:07', 'location': 'The Crownlands', 'subLocation': 'King'...",4
33,2014-04-27,Jaime entrusts a task to Brienne. Daenerys frees Meereen. Jon is given permission to lead a group of ...,/title/tt2972428/,4,Oathkeeper,"[King's Landing, Dragonstone, Dreadfort, Winterfell, The Wall, Meereen]","[{'sceneStart': '0:01:54', 'sceneEnd': '0:03:48', 'location': 'Meereen', 'characters': [{'name': 'Gre...",4
34,2014-05-04,Tommen is crowned King of the Seven Kingdoms. Cersei builds her case against Tyrion. Sansa and Lord B...,/title/tt3060856/,5,First of His Name,"[King's Landing, Dragonstone, Dreadfort, Winterfell, The Wall, Meereen]","[{'sceneStart': '0:01:55', 'sceneEnd': '0:02:54', 'location': 'The Crownlands', 'subLocation': 'King'...",4
35,2014-05-11,Tyrion's trial has come. Yara and her troops storm the Dreadfort to free Theon. Daenerys meets Hizdar...,/title/tt3060910/,6,The Laws of Gods and Men,"[King's Landing, Dreadfort, Winterfell, The Wall, Braavos, Meereen]","[{'sceneStart': '0:01:54', 'sceneEnd': '0:02:41', 'location': 'Braavos', 'characters': [{'name': 'Sta...",4
36,2014-05-18,Tyrion tries to find a champion. Daenerys sleeps with Daario. The Hound becomes wounded. Jon's advice...,/title/tt3060876/,7,Mockingbird,"[King's Landing, Dreadfort, Winterfell, The Wall, Braavos, Meereen]","[{'sceneStart': '0:01:54', 'sceneEnd': '0:04:56', 'location': 'The Crownlands', 'subLocation': 'King'...",4
37,2014-06-01,Theon helps Ramsay seize Moat Cailin. The wildlings attack Mole's Town. Sansa comes up with a story t...,/title/tt3060782/,8,The Mountain and the Viper,"[King's Landing, Moat Cailin, Winterfell, The Wall, Braavos, Meereen]","[{'sceneStart': '0:02:55', 'sceneEnd': '0:05:16', 'location': 'The Wall', 'subLocation': 'Mole's Town...",4
38,2014-06-08,The battle between the Night's Watch and the Wildlings has come.,/title/tt3060858/,9,The Watchers on the Wall,"[King's Landing, Moat Cailin, Winterfell, The Wall, Braavos, Meereen]","[{'sceneStart': '0:02:44', 'sceneEnd': '0:06:03', 'location': 'The Wall', 'subLocation': 'Castle Blac...",4
39,2014-06-15,"The Night's Watch story continues, with surprising developments. Daenerys experiences new consequence...",/title/tt3060860/,10,The Children,"[King's Landing, Moat Cailin, Winterfell, The Wall, Braavos, Meereen]","[{'sceneStart': '0:02:50', 'sceneEnd': '0:03:25', 'location': 'North of the Wall', 'subLocation': 'Th...",4


In [5]:
episodes = []
for i, ep in df.iterrows():
    season = ep['seasonNum']
    season = 'S' + str(season)
    episode = ep['episodeNum']
    episode = 'E' + str(episode).rjust(2, '0')
    episodes.append(season + episode)

In [6]:
personnages = {}
for ep in df['scenes']:
    for sc in ep:
        characters = sc['characters']
        for ch in characters:
            if ch['name'] in personnages:
                personnages[ch['name']] += 1
            else:
                personnages[ch['name']] = 1

personnages = pd.DataFrame(personnages, index=['appearances']).T
personnages = personnages[personnages['appearances'] > 50]

In [7]:
apparitions = pd.DataFrame(False, index=personnages.index, columns=episodes)

In [8]:
for i, ep in enumerate(df['scenes']):
    for sc in ep:
        if all(c not in sc.keys() for c in ['warg', 'flashback']):
            characters = sc['characters']
            for ch in characters:
                if ch['name'] in apparitions.index:
                    apparitions.loc[ch['name']].iloc[i] = True

In [9]:
apparitions

,S1E01,S1E02,S1E03,S1E04,S1E05,S1E06,S1E07,S1E08,S1E09,S1E10,...,S7E04,S7E05,S7E06,S7E07,S8E01,S8E02,S8E03,S8E04,S8E05,S8E06
Jon Snow,True,True,True,True,False,False,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
Bran Stark,True,True,True,True,True,True,False,True,False,True,...,True,True,False,True,True,True,True,True,False,True
Robb Stark,True,True,True,True,False,True,False,True,True,True,...,False,False,False,False,False,False,False,False,False,False
Eddard Stark,True,True,True,True,True,True,True,True,True,True,...,False,False,False,False,False,False,False,False,False,False
Catelyn Stark,True,True,True,True,True,True,False,True,True,True,...,False,False,False,False,False,False,False,False,False,False
Sansa Stark,True,True,True,True,True,True,False,True,True,True,...,True,True,True,True,True,True,True,True,False,True
Arya Stark,True,True,True,True,True,True,False,True,True,True,...,True,True,True,True,True,True,True,True,True,True
Theon Greyjoy,True,True,False,True,True,True,True,True,True,True,...,True,False,False,True,True,True,True,True,False,False
Ghost,True,True,False,True,False,False,True,True,False,True,...,False,False,False,False,False,True,True,True,False,True
Cersei Lannister,True,True,True,True,True,True,True,True,True,True,...,True,True,False,True,True,False,False,True,True,True


In [10]:
first = []
last = []
app_disp = pd.DataFrame(index=apparitions.index)
for i in apparitions.index:
    app = apparitions.loc[i][apparitions.loc[i]].index
    first.append(app[0])
    last.append(app[-1])

app_disp['first'] = first
app_disp['first'] = app_disp['first'].apply(lambda x: (int(x[1])-1)*10 + int(x[3:])-1)
app_disp['first'] = app_disp['first'].apply(lambda x: x if x<67 else x-3)
app_disp['last'] = last
app_disp['last'] = app_disp['last'].apply(lambda x: (int(x[1])-1)*10 + int(x[3:])-1)
app_disp['last'] = app_disp['last'].apply(lambda x: x if x<67 else x-3)

In [23]:
data = []
for name, ep in app_disp.iterrows():
    dictio = {}
    dictio['label'] = name
    dictio['data'] = [{'timeRange': [ep['first'], ep['last']], 'val': 0}]
    data.append(dictio)
    print(data)

[{'label': 'Jon Snow', 'data': [{'timeRange': [0, 72], 'val': 0}]}]
[{'label': 'Jon Snow', 'data': [{'timeRange': [0, 72], 'val': 0}]}, {'label': 'Bran Stark', 'data': [{'timeRange': [0, 72], 'val': 0}]}]
[{'label': 'Jon Snow', 'data': [{'timeRange': [0, 72], 'val': 0}]}, {'label': 'Bran Stark', 'data': [{'timeRange': [0, 72], 'val': 0}]}, {'label': 'Robb Stark', 'data': [{'timeRange': [0, 29], 'val': 0}]}]
[{'label': 'Jon Snow', 'data': [{'timeRange': [0, 72], 'val': 0}]}, {'label': 'Bran Stark', 'data': [{'timeRange': [0, 72], 'val': 0}]}, {'label': 'Robb Stark', 'data': [{'timeRange': [0, 29], 'val': 0}]}, {'label': 'Eddard Stark', 'data': [{'timeRange': [0, 9], 'val': 0}]}]
[{'label': 'Jon Snow', 'data': [{'timeRange': [0, 72], 'val': 0}]}, {'label': 'Bran Stark', 'data': [{'timeRange': [0, 72], 'val': 0}]}, {'label': 'Robb Stark', 'data': [{'timeRange': [0, 29], 'val': 0}]}, {'label': 'Eddard Stark', 'data': [{'timeRange': [0, 9], 'val': 0}]}, {'label': 'Catelyn Stark', 'data': [{

In [25]:
d3js = [{
    'group': "group",
    'data': data
}]

In [26]:
d3js

[{'group': 'group',
  'data': [{'label': 'Jon Snow', 'data': [{'timeRange': [0, 72], 'val': 0}]},
   {'label': 'Bran Stark', 'data': [{'timeRange': [0, 72], 'val': 0}]},
   {'label': 'Robb Stark', 'data': [{'timeRange': [0, 29], 'val': 0}]},
   {'label': 'Eddard Stark', 'data': [{'timeRange': [0, 9], 'val': 0}]},
   {'label': 'Catelyn Stark', 'data': [{'timeRange': [0, 28], 'val': 0}]},
   {'label': 'Sansa Stark', 'data': [{'timeRange': [0, 72], 'val': 0}]},
   {'label': 'Arya Stark', 'data': [{'timeRange': [0, 72], 'val': 0}]},
   {'label': 'Theon Greyjoy', 'data': [{'timeRange': [0, 70], 'val': 0}]},
   {'label': 'Ghost', 'data': [{'timeRange': [0, 72], 'val': 0}]},
   {'label': 'Cersei Lannister', 'data': [{'timeRange': [0, 72], 'val': 0}]},
   {'label': 'Jaime Lannister', 'data': [{'timeRange': [0, 72], 'val': 0}]},
   {'label': 'Joffrey Baratheon', 'data': [{'timeRange': [0, 32], 'val': 0}]},
   {'label': 'Sandor Clegane', 'data': [{'timeRange': [0, 71], 'val': 0}]},
   {'label': 

In [233]:
d3js[0]['data']

[]